### Code for object detection of animals namely- (elephant, giraffe, lion and zebra)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/qfgaohao/pytorch-ssd

Cloning into 'pytorch-ssd'...
remote: Enumerating objects: 812, done.
remote: Total 812 (delta 0), reused 0 (delta 0), pack-reused 812
Receiving objects: 100% (812/812), 1.05 MiB | 23.37 MiB/s, done.
Resolving deltas: 100% (544/544), done.


In [3]:
import sys
sys.path.append("/content/pytorch-ssd")

In [4]:
import numpy as np
import logging
import pathlib
import xml.etree.ElementTree as ET
import cv2
import os


class VOCDataset:

    def __init__(self, root, transform=None, target_transform=None, is_test=False, keep_difficult=False, label_file=None):
        """Dataset for VOC data.
        Args:
            root: the root of the VOC2007 or VOC2012 dataset, the directory contains the following sub-directories:
                Annotations, ImageSets, JPEGImages, SegmentationClass, SegmentationObject.
        """
        self.root = pathlib.Path(root)
        self.transform = transform
        self.target_transform = target_transform
        if is_test:
            image_sets_file = self.root / "ImageSets/Main/test.txt"
        else:
            image_sets_file = self.root / "ImageSets/Main/trainval.txt"
        self.ids = VOCDataset._read_image_ids(image_sets_file)
        self.keep_difficult = keep_difficult

        # if the labels file exists, read in the class names
        label_file_name = self.root / "labels.txt"

        if os.path.isfile(label_file_name):
            class_string = ""
            with open(label_file_name, 'r') as infile:
                for line in infile:
                    class_string += line.rstrip()

            # classes should be a comma separated list
            
            classes = class_string.split(',')
            # prepend BACKGROUND as first class
            classes.insert(0, 'BACKGROUND')
            classes  = [ elem.replace(" ", "") for elem in classes]
            self.class_names = tuple(classes)
            logging.info("VOC Labels read from file: " + str(self.class_names))

        else:
            logging.info("No labels file, using default VOC classes.")
            self.class_names = ('BACKGROUND',
            'elephant',
            'giraffe',
            'lion',
            'zebra')


        self.class_dict = {class_name: i for i, class_name in enumerate(self.class_names)}
      
    def __getitem__(self, index):
        
        image_id = self.ids[index]
        boxes, labels, is_difficult = self._get_annotation(image_id)
        if not self.keep_difficult:
            boxes = boxes[is_difficult == 0]
            labels = labels[is_difficult == 0]
        image = self._read_image(image_id)
        if self.transform:
            image, boxes, labels = self.transform(image, boxes, labels)
        if self.target_transform:
            boxes, labels = self.target_transform(boxes, labels)
       
        return image, boxes, labels

    def get_image(self, index):
        image_id = self.ids[index]
        image = self._read_image(image_id)
        if self.transform:
            image, _ = self.transform(image)
        return image

    def get_annotation(self, index):
        image_id = self.ids[index]
        return image_id, self._get_annotation(image_id)

    def __len__(self):
        return len(self.ids)

    @staticmethod
    def _read_image_ids(image_sets_file):
        ids = []
        with open(image_sets_file) as f:
            for line in f:
                ids.append(line.rstrip())
        return ids

    def _get_annotation(self, image_id):
        annotation_file = self.root / f"Annotations/{image_id}.xml"
        objects = ET.parse(annotation_file).findall("object")
        boxes = []
        labels = []
        is_difficult = []
        for object in objects:
            class_name = object.find('name').text.lower().strip()
            # we're only concerned with clases in our list
            if class_name in self.class_dict:
                bbox = object.find('bndbox')

                # VOC dataset format follows Matlab, in which indexes start from 0
                x1 = float(bbox.find('xmin').text) - 1
                y1 = float(bbox.find('ymin').text) - 1
                x2 = float(bbox.find('xmax').text) - 1
                y2 = float(bbox.find('ymax').text) - 1
                boxes.append([x1, y1, x2, y2])

                labels.append(self.class_dict[class_name])
                is_difficult_str = object.find('difficult').text
                is_difficult.append(0)
      
        return (np.array(boxes, dtype=np.float32),
                np.array(labels, dtype=np.int64),
                np.array(is_difficult, dtype=np.uint8))

    def _read_image(self, image_id):
        image_file = self.root / f"JPEGImages/{image_id}.jpg"
        image = cv2.imread(str(image_file))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

In [5]:
!wget -P models https://storage.googleapis.com/models-hao/mobilenet_v1_with_relu_69_5.pth

--2021-03-08 04:53:23--  https://storage.googleapis.com/models-hao/mobilenet_v1_with_relu_69_5.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12932699 (12M) [application/octet-stream]
Saving to: ‘models/mobilenet_v1_with_relu_69_5.pth’

mobilenet_v1_with_r 100%[===================>]  12.33M  73.5MB/s    in 0.2s    

2021-03-08 04:53:23 (73.5 MB/s) - ‘models/mobilenet_v1_with_relu_69_5.pth’ saved [12932699/12932699]



In [6]:
import os
import logging
import sys
import itertools

import torch
from torch.utils.data import DataLoader, ConcatDataset
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR

from vision.utils.misc import str2bool, Timer, freeze_net_layers, store_labels
from vision.ssd.ssd import MatchPrior
from vision.ssd.vgg_ssd import create_vgg_ssd
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_large_ssd_lite, create_mobilenetv3_small_ssd_lite
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite
#from vision.datasets.voc_dataset import VOCDataset
from vision.datasets.open_images import OpenImagesDataset
from vision.nn.multibox_loss import MultiboxLoss
from vision.ssd.config import vgg_ssd_config
from vision.ssd.config import mobilenetv1_ssd_config
from vision.ssd.config import squeezenet_ssd_config
from vision.ssd.data_preprocessing import TrainAugmentation, TestTransform


dataset_type = "voc"
datasets = ["/content/drive/MyDrive/projects_google_collab/object_detection/wild_life_object/data/train/", "/content/drive/MyDrive/projects_google_collab/object_detection/wild_life_object/data/val/"]
validation_dataset = "/content/drive/MyDrive/projects_google_collab/object_detection/wild_life_object/data/test"
balance_data = True
net = "mb1-ssd"
freeze_base_net = True
freeze_net = False
mb2_width_mult = 1.0

#params for SGD
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4
gamma = 0.1
base_net_lr = None
extra_layers_lr = None

# Params for loading pretrained basenet or checkpoints.
base_net = "/content/models/mobilenet_v1_with_relu_69_5.pth"
pretrained_ssd = False
resume = None

#Scheduler
scheduler = "cosine"
milestones = "80100"

#params for cosine Annealing
t_max = 200

#Train params
batch_size = 24
num_epochs = 200
num_workers = 4
validation_epochs = 5
debug_steps = 100
use_cuda = True
checkpoint_folder = 'models/'



logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
#args = parser.parse_args()
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() and use_cuda else "cpu")

if use_cuda and torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    logging.info("Use Cuda.")


def train(loader, net, criterion, optimizer, device, debug_steps=100, epoch=-1):
    net.train(True)
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    for i, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        confidence, locations = net(images)
        regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)  # TODO CHANGE BOXES
        loss = regression_loss + classification_loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
        if i and i % debug_steps == 0:
            avg_loss = running_loss / debug_steps
            avg_reg_loss = running_regression_loss / debug_steps
            avg_clf_loss = running_classification_loss / debug_steps
            logging.info(
                f"Epoch: {epoch}, Step: {i}, " +
                f"Average Loss: {avg_loss:.4f}, " +
                f"Average Regression Loss {avg_reg_loss:.4f}, " +
                f"Average Classification Loss: {avg_clf_loss:.4f}"
            )
            running_loss = 0.0
            running_regression_loss = 0.0
            running_classification_loss = 0.0


def test(loader, net, criterion, device):
    net.eval()
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    num = 0
    for _, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)
        num += 1

        with torch.no_grad():
            confidence, locations = net(images)
            regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)
            loss = regression_loss + classification_loss

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
    return running_loss / num, running_regression_loss / num, running_classification_loss / num


if __name__ == '__main__':
    timer = Timer()

    #logging.info(args)
    if net == 'vgg16-ssd':
        create_net = create_vgg_ssd
        config = vgg_ssd_config
    elif net == 'mb1-ssd':
        create_net = create_mobilenetv1_ssd
        config = mobilenetv1_ssd_config
    elif net == 'mb1-ssd-lite':
        create_net = create_mobilenetv1_ssd_lite
        config = mobilenetv1_ssd_config
    elif net == 'sq-ssd-lite':
        create_net = create_squeezenet_ssd_lite
        config = squeezenet_ssd_config
    elif net == 'mb2-ssd-lite':
        create_net = lambda num: create_mobilenetv2_ssd_lite(num, width_mult=args.mb2_width_mult)
        config = mobilenetv1_ssd_config
    elif net == 'mb3-large-ssd-lite':
        create_net = lambda num: create_mobilenetv3_large_ssd_lite(num)
        config = mobilenetv1_ssd_config
    elif net == 'mb3-small-ssd-lite':
        create_net = lambda num: create_mobilenetv3_small_ssd_lite(num)
        config = mobilenetv1_ssd_config
    else:
        logging.fatal("The net type is wrong.")
        parser.print_help(sys.stderr)
        sys.exit(1)
    train_transform = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
    target_transform = MatchPrior(config.priors, config.center_variance,
                                  config.size_variance, 0.5)

    test_transform = TestTransform(config.image_size, config.image_mean, config.image_std)

    logging.info("Prepare training datasets.")
    datasets_list = []
    
    for dataset_path in datasets:
        #dataset_path = datasets + "/"+str(filename)
        if dataset_type == 'voc':
            print(dataset_path)
            dataset = VOCDataset(dataset_path, transform=train_transform,
                                 target_transform=target_transform)
            label_file = os.path.join(checkpoint_folder, "voc-model-labels.txt")
            store_labels(label_file, dataset.class_names)
            num_classes = len(dataset.class_names)
        elif dataset_type == 'open_images':
            dataset = OpenImagesDataset(dataset_path,
                 transform=train_transform, target_transform=target_transform,
                 dataset_type="train", balance_data=balance_data)
            label_file = os.path.join(checkpoint_folder, "open-images-model-labels.txt")
            store_labels(label_file, dataset.class_names)
            logging.info(dataset)
            num_classes = len(dataset.class_names)

        else:
            raise ValueError(f"Dataset type {dataset_type} is not supported.")
        datasets_list.append(dataset)
    logging.info(f"Stored labels into file {label_file}.")
    train_dataset = ConcatDataset(datasets_list)
    logging.info("Train dataset size: {}".format(len(train_dataset)))
    train_loader = DataLoader(train_dataset, batch_size,
                              num_workers=num_workers,
                              shuffle=True)
    logging.info("Prepare Validation datasets.")
    if dataset_type == "voc":
        val_dataset = VOCDataset(validation_dataset, transform=test_transform,
                                 target_transform=target_transform, is_test=True)
    elif dataset_type == 'open_images':
        val_dataset = OpenImagesDataset(dataset_path,
                                        transform=test_transform, target_transform=target_transform,
                                        dataset_type="test")
        logging.info(val_dataset)
    logging.info("validation dataset size: {}".format(len(val_dataset)))

    val_loader = DataLoader(val_dataset, batch_size,
                            num_workers=num_workers,
                            shuffle=False)
    logging.info("Build network.")
    net_name = net
    net = create_net(num_classes)
    min_loss = -10000.0
    last_epoch = -1

    base_net_lr = base_net_lr if base_net_lr is not None else lr
    extra_layers_lr = extra_layers_lr if extra_layers_lr is not None else lr
    if freeze_base_net:
        logging.info("Freeze base net.")
        freeze_net_layers(net.base_net)
        params = itertools.chain(net.source_layer_add_ons.parameters(), net.extras.parameters(),
                                 net.regression_headers.parameters(), net.classification_headers.parameters())
        params = [
            {'params': itertools.chain(
                net.source_layer_add_ons.parameters(),
                net.extras.parameters()
            ), 'lr': extra_layers_lr},
            {'params': itertools.chain(
                net.regression_headers.parameters(),
                net.classification_headers.parameters()
            )}
        ]
    elif freeze_net:
        freeze_net_layers(net.base_net)
        freeze_net_layers(net.source_layer_add_ons)
        freeze_net_layers(net.extras)
        params = itertools.chain(net.regression_headers.parameters(), net.classification_headers.parameters())
        logging.info("Freeze all the layers except prediction heads.")
    else:
        params = [
            {'params': net.base_net.parameters(), 'lr': base_net_lr},
            {'params': itertools.chain(
                net.source_layer_add_ons.parameters(),
                net.extras.parameters()
            ), 'lr': extra_layers_lr},
            {'params': itertools.chain(
                net.regression_headers.parameters(),
                net.classification_headers.parameters()
            )}
        ]

    timer.start("Load Model")
    if resume:
        logging.info(f"Resume from the model {resume}")
        net.load(resume)
    elif base_net:
        logging.info(f"Init from base net {base_net}")
        net.init_from_base_net(base_net)
    elif pretrained_ssd:
        logging.info(f"Init from pretrained ssd {pretrained_ssd}")
        net.init_from_pretrained_ssd(pretrained_ssd)
    logging.info(f'Took {timer.end("Load Model"):.2f} seconds to load the model.')

    net.to(DEVICE)

    criterion = MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3,
                             center_variance=0.1, size_variance=0.2, device=DEVICE)
    optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum,
                                weight_decay=weight_decay)
    logging.info(f"Learning rate: {lr}, Base net learning rate: {base_net_lr}, "
                 + f"Extra Layers learning rate: {extra_layers_lr}.")

    if scheduler == 'multi-step':
        logging.info("Uses MultiStepLR scheduler.")
        milestones = [int(v.strip()) for v in milestones.split(",")]
        scheduler = MultiStepLR(optimizer, milestones=milestones,
                                                     gamma=0.1, last_epoch=last_epoch)
    elif scheduler == 'cosine':
        logging.info("Uses CosineAnnealingLR scheduler.")
        scheduler = CosineAnnealingLR(optimizer, t_max, last_epoch=last_epoch)
    else:
        logging.fatal(f"Unsupported Scheduler: {scheduler}.")
        parser.print_help(sys.stderr)
        sys.exit(1)

    logging.info(f"Start training from epoch {last_epoch + 1}.")
    for epoch in range(last_epoch + 1, num_epochs):
        scheduler.step()
        train(train_loader, net, criterion, optimizer,
              device=DEVICE, debug_steps=debug_steps, epoch=epoch)
        
        if epoch % validation_epochs == 0 or epoch == num_epochs - 1:
            val_loss, val_regression_loss, val_classification_loss = test(val_loader, net, criterion, DEVICE)
            
            logging.info(
                f"Epoch: {epoch}, " +
                f"Validation Loss: {val_loss:.4f}, " +
                f"Validation Regression Loss {val_regression_loss:.4f}, " +
                f"Validation Classification Loss: {val_classification_loss:.4f}"
            )

            #print("-----> {}".format(net, epoch, val_loss))
            model_path = os.path.join(checkpoint_folder, "{}-Epoch-{}-Loss-{}.pth".format(str(net_name), epoch, val_loss))
            net.save(model_path)
            logging.info(f"Saved model {model_path}")

2021-03-08 04:53:29,181 - root - INFO - Use Cuda.
2021-03-08 04:53:29,270 - root - INFO - Prepare training datasets.
/content/drive/MyDrive/projects_google_collab/object_detection/wild_life_object/data/train/
2021-03-08 04:53:30,342 - root - INFO - No labels file, using default VOC classes.
/content/drive/MyDrive/projects_google_collab/object_detection/wild_life_object/data/val/
2021-03-08 04:53:31,044 - root - INFO - No labels file, using default VOC classes.
2021-03-08 04:53:31,048 - root - INFO - Stored labels into file models/voc-model-labels.txt.
2021-03-08 04:53:31,048 - root - INFO - Train dataset size: 128
2021-03-08 04:53:31,050 - root - INFO - Prepare Validation datasets.
2021-03-08 04:53:31,829 - root - INFO - No labels file, using default VOC classes.
2021-03-08 04:53:31,829 - root - INFO - validation dataset size: 4
2021-03-08 04:53:31,830 - root - INFO - Build network.
2021-03-08 04:53:31,914 - root - INFO - Freeze base net.
2021-03-08 04:53:31,917 - root - INFO - Init fr

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarnin

2021-03-08 04:54:36,987 - root - INFO - Epoch: 0, Validation Loss: 10.1072, Validation Regression Loss 2.7279, Validation Classification Loss: 7.3793
2021-03-08 04:54:37,070 - root - INFO - Saved model models/mb1-ssd-Epoch-0-Loss-10.107227325439453.pth


/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, y

2021-03-08 04:55:48,704 - root - INFO - Epoch: 5, Validation Loss: 9.0521, Validation Regression Loss 2.5124, Validation Classification Loss: 6.5397
2021-03-08 04:55:48,774 - root - INFO - Saved model models/mb1-ssd-Epoch-5-Loss-9.052132606506348.pth


...

2021-03-08 05:40:27,010 - root - INFO - Epoch: 190, Validation Loss: 4.9974, Validation Regression Loss 1.6576, Validation Classification Loss: 3.3398
2021-03-08 05:40:27,079 - root - INFO - Saved model models/mb1-ssd-Epoch-190-Loss-4.997366905212402.pth


/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, y

2021-03-08 05:41:41,374 - root - INFO - Epoch: 195, Validation Loss: 4.9007, Validation Regression Loss 1.6390, Validation Classification Loss: 3.2618
2021-03-08 05:41:41,445 - root - INFO - Saved model models/mb1-ssd-Epoch-195-Loss-4.90071964263916.pth


/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/content/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, y

2021-03-08 05:42:38,893 - root - INFO - Epoch: 199, Validation Loss: 4.9836, Validation Regression Loss 1.6509, Validation Classification Loss: 3.3327
2021-03-08 05:42:38,963 - root - INFO - Saved model models/mb1-ssd-Epoch-199-Loss-4.983603000640869.pth


In [ ]:
from vision.ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd, create_mobilenetv1_ssd_predictor
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite, create_mobilenetv1_ssd_lite_predictor
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite, create_squeezenet_ssd_lite_predictor
from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite, create_mobilenetv2_ssd_lite_predictor
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_large_ssd_lite, create_mobilenetv3_small_ssd_lite
from vision.utils.misc import Timer     
import cv2
import sys
from google.colab.patches import cv2_imshow

def inference(net_type, model_path, label_path, input_file, output_file):

  cap = cv2.VideoCapture(input_file)  # capture from file
  left = cap.set(3, 1920)
  right = cap.set(4, 1080)
  fourcc_codec = cv2.VideoWriter_fourcc(*'XVID')
  fps = 30.0
  capture = (int(cap.get(3)), int(cap.get(4)))
  videowriter = cv2.VideoWriter(output_file, fourcc_codec, fps, capture)
  timer = Timer()
  class_names = [name.strip() for name in open(label_path).readlines()]
  num_classes = len(class_names)


  if net_type == 'vgg16-ssd':
      net = create_vgg_ssd(len(class_names), is_test=True)
  elif net_type == 'mb1-ssd':
      net = create_mobilenetv1_ssd(len(class_names), is_test=True)
  elif net_type == 'mb1-ssd-lite':
      net = create_mobilenetv1_ssd_lite(len(class_names), is_test=True)
  elif net_type == 'mb2-ssd-lite':
      net = create_mobilenetv2_ssd_lite(len(class_names), is_test=True)
  elif net_type == 'mb3-large-ssd-lite':
      net = create_mobilenetv3_large_ssd_lite(len(class_names), is_test=True)
  elif net_type == 'mb3-small-ssd-lite':
      net = create_mobilenetv3_small_ssd_lite(len(class_names), is_test=True)
  elif net_type == 'sq-ssd-lite':
      net = create_squeezenet_ssd_lite(len(class_names), is_test=True)
  else:
      print("The net type is wrong. It should be one of vgg16-ssd, mb1-ssd and mb1-ssd-lite.")
      sys.exit(1)
  net.load(model_path)

  if net_type == 'vgg16-ssd':
      predictor = create_vgg_ssd_predictor(net, candidate_size=200)
  elif net_type == 'mb1-ssd':
      predictor = create_mobilenetv1_ssd_predictor(net, candidate_size=200)
  elif net_type == 'mb1-ssd-lite':
      predictor = create_mobilenetv1_ssd_lite_predictor(net, candidate_size=200)
  elif net_type == 'mb2-ssd-lite' or net_type == "mb3-large-ssd-lite" or net_type == "mb3-small-ssd-lite":
      predictor = create_mobilenetv2_ssd_lite_predictor(net, candidate_size=200)
  elif net_type == 'sq-ssd-lite':
      predictor = create_squeezenet_ssd_lite_predictor(net, candidate_size=200)
  else:
      print("The net type is wrong. It should be one of vgg16-ssd, mb1-ssd and mb1-ssd-lite.")
      sys.exit(1)

  while True:
      ret, orig_image = cap.read()
      if orig_image is None:
          continue
      image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
      timer.start()
      boxes, labels, probs = predictor.predict(image, 10, 0.4)
      interval = timer.end()
      print('Time: {:.2f}s, Detect Objects: {:d}.'.format(interval, labels.size(0)))
      for i in range(boxes.size(0)):
        try:
          box = boxes[i, :]
          label = f"{class_names[labels[i]]}: {probs[i]:.2f}"
          cv2.rectangle(orig_image, (box[0], box[1]), (box[2], box[3]), (255, 255, 0), 4)

          cv2.putText(orig_image, label,
                      (box[0]+20, box[1]+40),
                      cv2.FONT_HERSHEY_SIMPLEX,
                      1,  # font scale
                      (255, 0, 255),
                      2)  # line type
        
          videowriter.write(orig_image)
          
        except Exception as e:
          print(e)
          videowriter.write(orig_image)
          continue

      if cv2.waitKey(1) & 0xFF == ord('q'):
          break
  cap.release()
  videowriter.release()
  cv2.destroyAllWindows()


if __name__ == "__main__":
  net_type = "mb1-ssd"
  model_path = "/content/models/mb1-ssd-Epoch-199-Loss-4.983603000640869.pth" 
  label_path = "/content/models/voc-model-labels.txt"
  input_file = "/content/drive/MyDrive/projects_google_collab/object_detection/wild_life_object/data/Input_trial_video.mp4"
  output_file = "/content/drive/MyDrive/projects_google_collab/object_detection/wild_life_object/data/output_video.avi"

  #start infrencing
  inference(net_type, model_path, label_path, input_file, output_file)
  
  print("#######################################################################################")
  print("#                                                                                     #")
  print("#                                                                                     #")
  print("#                    PROCESS COMPLETED!                                               #")
  print("#       your video is ready and saved as output_video.avi                             #")
  print("#                                                                                     #")
  print("#                                                                                     #")
  print("#######################################################################################")

Streaming output truncated to the last 5000 lines.
Inference time:  0.006727933883666992
Time: 0.01s, Detect Objects: 0.
Inference time:  0.006711006164550781
Time: 0.01s, Detect Objects: 0.
Inference time:  0.005069732666015625
Time: 0.01s, Detect Objects: 2.
Inference time:  0.007169961929321289
Time: 0.01s, Detect Objects: 1.
Inference time:  0.00871133804321289
Time: 0.01s, Detect Objects: 1.
Inference time:  0.008857250213623047
Time: 0.02s, Detect Objects: 2.
Inference time:  0.007548332214355469
Time: 0.01s, Detect Objects: 1.
Inference time:  0.009081125259399414
Time: 0.01s, Detect Objects: 2.
Inference time:  0.008503437042236328
Time: 0.02s, Detect Objects: 2.
Inference time:  0.008009910583496094
Time: 0.01s, Detect Objects: 2.
Inference time:  0.0062007904052734375
Time: 0.01s, Detect Objects: 1.
Inference time:  0.0051000118255615234
Time: 0.01s, Detect Objects: 2.
Inference time:  0.005190372467041016
Time: 0.01s, Detect Objects: 1.
Inference time:  0.006761074066162109
